# क्लाउडमधील डेटा सायन्स: "Azure ML SDK" मार्ग

## परिचय

या नोटबुकमध्ये, आपण Azure ML SDK चा वापर करून मॉडेल प्रशिक्षण, तैनात आणि वापरण्याची प्रक्रिया शिकणार आहोत.

पूर्व-आवश्यकता:
1. तुम्ही Azure ML workspace तयार केले आहे.
2. तुम्ही [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) Azure ML मध्ये लोड केले आहे.
3. तुम्ही ही नोटबुक Azure ML Studio मध्ये अपलोड केली आहे.

पुढील पायऱ्या आहेत:

1. विद्यमान Workspace मध्ये एक Experiment तयार करा.
2. एक Compute cluster तयार करा.
3. डेटासेट लोड करा.
4. AutoMLConfig वापरून AutoML कॉन्फिगर करा.
5. AutoML experiment चालवा.
6. निकालांचा अभ्यास करा आणि सर्वोत्तम मॉडेल मिळवा.
7. सर्वोत्तम मॉडेल नोंदणी करा.
8. सर्वोत्तम मॉडेल तैनात करा.
9. Endpoint वापरा.

## Azure Machine Learning SDK-साठी विशिष्ट आयात


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## कार्यक्षेत्र प्रारंभ करा  
संचयित कॉन्फिगरेशनमधून कार्यक्षेत्र ऑब्जेक्ट प्रारंभ करा. .\config.json येथे कॉन्फिगरेशन फाइल उपलब्ध असल्याची खात्री करा.  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML प्रयोग तयार करा

आपण नुकत्याच प्रारंभ केलेल्या कार्यक्षेत्रात 'aml-experiment' नावाचा एक प्रयोग तयार करूया.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## संगणन क्लस्टर तयार करा  
तुमच्या AutoML रनसाठी तुम्हाला [संगणन लक्ष्य](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) तयार करणे आवश्यक आहे.  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## डेटा  
आपण डेटासेट Azure ML वर अपलोड केला आहे याची खात्री करा आणि की डेटासेटच्या नावासारखीच आहे.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## सर्वोत्तम मॉडेल तैनात करा

सर्वोत्तम मॉडेल तैनात करण्यासाठी खालील कोड चालवा. तुम्ही Azure ML पोर्टलमध्ये तैनातीची स्थिती पाहू शकता. या टप्प्याला काही मिनिटे लागू शकतात.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## एन्डपॉइंट वापरा
तुम्ही खालील इनपुट नमुन्यात इनपुट्स जोडू शकता.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
